In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from scipy.signal import butter, filtfilt
from scipy.io import loadmat
import wfdb
from wfdb import rdrecord

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from train_ecg_vae_reutilice import ECGDataset, ECG_VAE, loss_fn

In [ ]:
# PTB_DIR = 'data/ptb-xl'
# CH_DIR  = 'data/ChapmanShaoxing'
# # Hyperparams
# batch_size = 16
# epochs = 16
# lr = 1e-3
# z_dim = 16
# seq_len = 5000

# # Dataset & Loader
# ds = ECGDataset(PTB_DIR, CH_DIR, sample_length=seq_len)
# loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=4)

# # Model, optim
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# vae = ECG_VAE(z_dim=z_dim, seq_len=seq_len).to(device)
# opt = optim.Adam(vae.parameters(), lr=lr)

# # Train
# for ep in range(1, epochs+1):
#     vae.train()
#     tot_loss = 0
#     for batch in loader:
#         x = batch.to(device)
#         x_hat, mu, logv = vae(x)
#         loss, recon, kld = loss_fn(x, x_hat, mu, logv)
#         opt.zero_grad(); loss.backward(); opt.step()
#         tot_loss += loss.item()
#     print(f"Epoch {ep}/{epochs} - Loss: {tot_loss/len(loader):.4f} (Recon {recon:.4f}, KLD {kld:.4f})")
# # Guardar
# torch.save(vae.state_dict(), 'ecg_vae.pth')
# print("Modelo guardado en ecg_vae.pth")


Epoch 1/16 - Loss: 9617397089213.7285 (Recon 0.2260, KLD 126.6771)
Epoch 2/16 - Loss: 147356917007036704184336384.0000 (Recon 0.1953, KLD 165.2787)
Epoch 3/16 - Loss: 3747946700.5763 (Recon 0.1680, KLD 143.4911)
Epoch 4/16 - Loss: 1143578589801997696.0000 (Recon 0.2545, KLD 156.9068)


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from scipy.signal import butter, filtfilt
from scipy.io import loadmat
import wfdb
from wfdb import rdrecord

In [ ]:
from VAE import ECGDataset, ECG_VAE, loss_fn

In [ ]:
PTB_DIR='data/ptb-xl'; CH_DIR='data/ChapmanShaoxing'
batch=16; epochs=15; lr=1e-3; z_dim=16; seq_len=5000
# dataset completo
ds=ECGDataset(PTB_DIR,CH_DIR,seq_len)
# split: 80% train, 10% val, 10% test
n=len(ds); n_test=int(0.1*n); n_val=int(0.1*n); n_train=n-n_val-n_test
ds_train, ds_val, ds_test = random_split(ds, [n_train,n_val,n_test])
loader_tr=DataLoader(ds_train,batch,shuffle=True,num_workers=4)
loader_val=DataLoader(ds_val,batch,shuffle=False,num_workers=2)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vae=ECG_VAE(z_dim=z_dim, seq_len=seq_len).to(device)
opt=optim.Adam(vae.parameters(), lr=lr)

best_val=float('inf')
for ep in range(1,epochs+1):
    vae.train(); train_loss=0
    for xb in loader_tr:
        xb=xb.to(device)
        xh, mu, lv=vae(xb)
        loss,_,_ = loss_fn(xb,xh,mu,lv)
        opt.zero_grad(); loss.backward();
        nn.utils.clip_grad_norm_(vae.parameters(),1.0)
        opt.step(); train_loss+=loss.item()
    train_loss/=len(loader_tr)
    # validación
    vae.eval(); val_loss=0
    with torch.no_grad():
        for xb in loader_val:
            xb=xb.to(device)
            xh,mu,lv=vae(xb)
            loss,_,_=loss_fn(xb,xh,mu,lv)
            val_loss+=loss.item()
    val_loss/=len(loader_val)
    print(f"Epoch {ep}/{epochs} - Train L: {train_loss:.4f}  Val L: {val_loss:.4f}")
    if val_loss<best_val:
        best_val=val_loss
        torch.save(vae.state_dict(),'ecg_vae_best.pth')
print("Entrenamiento completo. Mejor modelo en ecg_vae_best.pth")
# test: MAE sobre ds_test
from torch.utils.data import DataLoader
loader_te=DataLoader(ds_test,batch,shuffle=False)
tot_mae=[]
vae.load_state_dict(torch.load('ecg_vae_best.pth',map_location=device))
vae.eval()
with torch.no_grad():
    for xb in loader_te:
        xb=xb.to(device)
        xh,_,_=vae(xb)
        mae=torch.abs(xh-xb).mean(dim=(1,2)).cpu().numpy()
        tot_mae.extend(mae.tolist())
print(f"MAE promedio normales test: {np.mean(tot_mae):.4f}")
# ds_test contiene sólo normales. Anómalos se evalúan en script aparte.